<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Handcrafting-Ethereum-vocabulary" data-toc-modified-id="Handcrafting-Ethereum-vocabulary-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Handcrafting Ethereum vocabulary</a></span><ul class="toc-item"><li><span><a href="#Proof-of-Concept" data-toc-modified-id="Proof-of-Concept-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Proof of Concept</a></span></li><li><span><a href="#Formatting-for-JSON" data-toc-modified-id="Formatting-for-JSON-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Formatting for JSON</a></span></li></ul></li></ul></div>

In [1]:
import pandas as pd
import spacy
from namedentities import named_entities # HTML escaping

from gensim.utils import simple_preprocess
from gensim.models import Phrases
from gensim.models.phrases import Phraser

/usr/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# Download the model with 'python -m spacy download en_core_web_lg --user'
# Note: this is a 800MB model
#       'en_core_web_sm', 29MB can be used as alternative
#       see https://spacy.io/models/en#section-en_core_web_lg
nlp = spacy.load('en_core_web_lg')

In [3]:
df = pd.read_excel('interviews_20180710.xls')

In [4]:
df

,Name,How do you handle smart contract verification and security?,Other bounties?,Who are you and what are you working on?,What are the tools/libraries/frameworks you use?,What are your biggest frustrations?,How do you handle testing?,What tools don’t exist at the moment?,Other domain specific questions?,What was the hardest part about learning to develop with Ethereum?,What are you most excited about in the short term?,Was anything easier than expected?,Who are the other people you think we should talk to?,What are the best educational resources?,Are there any other questions we should be asking?
0,﻿Fabio Berger + Remco Bloemen,Code audits before deployment\nWant to add a f...,Rest of the bounties from Remco\nState-watchin...,0x - Decentralized exchange protocol. It is a ...,EthereumJS / truffle hardwallet provider / Typ...,Getting a simple experimental environment up i...,Testrpc has gotten better ––\nIncreases the sp...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,﻿Leo Logvinov,NaN,There is a limit on local variable numbers (ha...,"Started in blockchain 2 years ago in Berlin, w...",NaN,"Event watching - unreliable, no support for ba...",NaN,Prettier type plugin for solidity. I don’t hav...,What has your experience been like working on ...,NaN,NaN,NaN,NaN,NaN,NaN
2,﻿Axel Ericsson,Human thinking and auditing.\nWill find some e...,Breaking out a javascript library for web page...,I have built 1Protocol\nIt lets smart contract...,Just web3JS and the python raiden client,NaN,"Truffle suite. Before that, built our own unit...",There is no tooling or anything related to sta...,NaN,"Solidity is changing so quickly that, as an ex...",Raiden is the golden egg in the space. The exp...,Writing smart contracts is surprisingly neat. ...,NaN,NaN,NaN
3,﻿Mike Goldin,NaN,Pretty cool to have more languages for the EVM...,Software developer at Consensys\nKnown for Tok...,"Truffle, useful as a testing framework.\nSolid...",Truffle’s debugger is a bit disappointing. Wor...,Writes tests in Javascript\nSol Coverage tool\...,Fuzz Testing and formal verification desired.,NaN,Hardest part is learning to develop in a block...,Excited for Casper\nApplications implemented i...,What’s really easy is programming money.\nOn b...,"Infura team, client development\nSpankchain - ...",Solidity Docs\nUses yellow paper gass lookup t...,NaN
4,﻿Oleksii,"We have written our own tools which tell us, f...",NaN,Started working with smart contracts in early ...,NaN,Our original vision was to do everything – tes...,NaN,NaN,What are other great tools you have used?\nEth...,NaN,NaN,NaN,NaN,NaN,NaN
5,﻿Brett Sun,Human audits only right now.\nTrailofbits is r...,NaN,Working on Aragon entirely.\nThe end goal is t...,Four categories:\nEVM and lower level stuff:\n...,My biggest general frustration comes with the ...,In addition to the above we also use solidity-...,A nice debugger! Please…\nMore infrastructure ...,NaN,NaN,NaN,NaN,NaN,Google + Github is my go to.\nNot enough has b...,NaN
6,﻿Jorge Izquierdo,NaN,Infura that is more decentralized. A full-node...,Aragon - Decentralized Governance platform\nWe...,NaN,NaN,NaN,NaN,Tell us more about getting deep into the EVM?\...,NaN,NaN,NaN,NaN,NaN,NaN
7,﻿Jack Peterson and Sparkle,Pay others to think about it - no process yet....,NaN,NaN,Back-end: Visual Studio Code with docker image...,Lack of a debugger - by far the biggest issue....,NaN,Setting breakpoints in tests!\nSalesforce Deve...,Let’s talk specifically about UI issues / scal...,What steps do I need to follow to write these ...,Raiden is coming shortly and will be very cool...,NaN,NaN,NaN,NaN
8,﻿Joey Krug,"Yeah, encourage everyone to use auditors. Lots...",NaN,Co-Chief Investment Officer at Pantera Capital...,"At Augur, stack is different as it’s a bit old...","At the end of the day, it all comes down to th...",Run against pyethereum. Test suite is written ...,"My answers changed over a time, 1 year ago it ...",Would be useful if the Ethereum project could ...,NaN,"In the short term, he’

In [5]:
df.columns = [
    'name',              # Name
    'smart_contract',    # How do you handle smart contract verif & security?
    'bounties',          # Other bounties
    'who_what',          # Who are you and what are you working on?
    'tooling',           # What are the tools/libraries/frameworks you use?
    'frustrations',      # What are your biggest frustrations?
    'testing',           # How do you handle testing?
    'missing_tools',     # What tools don’t exist at the moment?
    'domain_questions',  # Other domain specific questions?
    'hardest_part',      # What was the hardest part to develop with Ethereum?
    'excited_about',     # What are you most excited about in the short term?
    'easier_expected',   # Was anything easier than expected?
    'people_talk_to',    # Who do you think we should talk to?
    'best_resources',    # What are the best educational resources?
    'questions_to_ask'   # Are there any other questions we should be asking?
]

df.fillna('', inplace = True)

In [6]:
# For some reason Python or Excel adds an invisible unicode \ufeff character.
df['name'] = df['name'].str.replace(u'\ufeff', '')

# Handcrafting Ethereum vocabulary

In [7]:
eth_keywords = {
    # Who are you and what are you working on?
    '0x',
    'decentralized_exchange',
    'dex',
    'decentralization',
    'smart_contract',
    'solidity',
    'truffle',
    '1protocol',
    'proof_of_stake',
    'proof_stake',
    'pos',
    'mining_pool',
    'staking_pool',
    'signature',
    'token',
    'raiden',
    'api',
    'client',
    'consensys',
    'token_curated_registries',
    'tcr',
    'adchain',
    'onboarding',
    'test',
    'ambisafe',
    'scalable',
    'platform',
    'devops',
    'javascript',
    'js',
    'tx',
    'transactions',
    'parity',
    'deployment',
    'gas',
    'aragon',
    'aragon_os',
    'organization',
    'network',
    'permission',
    'governance',
    'acl',
    'access',
    'dapp',
    'backend',
    'protocol',
    'stack',
    'trustless',
    'metamask',
    'augur',
    'pantera_capital',
    'bounty',
    'bounties_network',
    'infura',
    'ethereum',
    'ipfs',
    'geth',
    'evm',
    'aion',
    'shyft',
    'web3',
    'merkle_tree',
    'audit',
    'zeppelin',
    'ares',
    'governx',
    'on-chain',
    'off-chain',
    'ico',
    'developer',
    'pm',
    'community',
    'content',
    'course',
    'resource',
    'mentor',
    'blockchain',
    'context',
    'history',
    'mindset',
    'ganache',
    'remix',
    'react',
    'react_native',
    'oraclize',
    'lll',
    'vyper',
    'eip',
    'ethereum_improvement_proposal',
    'security',
    'cryptokitties',
    'sharding',
    'dapphub',
    'compiler',
    'formal_verification',
    'formal_proof',
    'safe',
    'engineer',
    'tooling',
    'erc20',
    'fungible',
    'nft_token',
    'ipns',
    'auction',
    'ethalarm',
    'browser',
    'crypto',
    'cryptographic',
    'independent',
    'verifier',
    'investor',
    'decentraland',
    'open',
    'dharma',
    'library',
    'wrapper',
    'mist',
    'open-source',
    'colony',
    'status_open_bounties',
    'gitcoin',
    'status',
    'ens',
    'ui',
    'static_analysis',
    'documentation',
    'ethmoji',
    'openzeppelin',
    'ethwaterloo',
    'ethdenver',
    'ethvigil',
    'rest',
    'integration',
    'email',
    'slack',
    'vm',
    'design',
    'testrpc',
    'trezor',
    'rpc',
    'ewasm',
    'research',
    'plasma',
    'snarks',
    'starks',
    'zksnarks',
    'zkstarks',
    'truebit',
    'ethereum_wallet',
    'web3.js',
    'ERC725',
    'network',
    'p2p',
    'analysis',
    'hack',
    'regulatory',
    'giveth',
    'pledge',
    'chat',
    'discovery',
    'liquid',
    'dappnode',
    'makerdao',
    'identity',
    'gnosis',
    'ddex',
    'wallet',
    'hellogold',
    'horizon_blockchain',
    'games',
    'fair',
    'cross-chain',
    'finance',
    'swap',
    'contract',
    'solidify',
    'knowledge',
    'kauri',
    'tutorial',
    'dispute_resolution',
    'arbitration',
    'dao',
    'incentive',
    'decision',
    'production',
    'payment',
    'micro_payment',
    'ethereumjs',
    'kyokan',
    'counterfactual',
    'state_channel',
    'ethglobal',
    'database',
    'leroy',
    'steemit',
    'video',
    'infrastructure',
    'ethdeploy',
    'cryptozombies',
    'ethfiddle',
    'digital_asset',
    'crowdsale',
    'modular.network',
    'ethpm',
    'eal',
    'ethereum_address_lookup',
    'chrome extension',
    'phishing',
    'bot',
    'myetherwallet',
    'mew',
    'mycrypto',
    'bridge',
    'layer_2',
    'sidechain',
    'ethereum_permission_client',
    'wasm',
    'kovan',
    'trading',
    'token_contract',
    'consensus',
    'algorithm',
    'contract',
    'polymath',
    'ethereum_alarm_clock',
    'dagger',
    'multisig',
    'polychain',
    'trinity',
    'py-evm',
    'web3.py',
    'usability',
    'quickblocks',
    'embark',
    'samsara',
    'raffle',
    'escrow',
    'auction',
    'collateral',
    'portfolio',
    'slockit',
    'iot',
    'sourccred',
    'filecoin',
    'spankchain',
    'spank',
    'adtoken',
    'dao',
    'betting',
    'zocrates',
    'sweetbridge',
    'the_graph',
    'json_rpc',
    'jsonrpc',
    'graphql',
    'voting_system',
    'trustory',
    'tokenomics',
    'cryptoeconomic',
    'coinbase',
    'ujo_music',
    'virtuepoker',
    'monax',
    'maker',
    'vulcanizedb',
    'etl',
    'sql',
    'tendermint',
    'lightning'
    'federated_exchange',
    'week_in_ethereum',
    'week_ethereum'
    'newsletter',
    'r/ethereum',
    'moderator',
    'ERC721',
    'xlnt',
    'gnarly',
    'redis',
    'eip0',
    'ethereum magicians',
    'coworking',
    'crypto_nyc',
    'odin',
    'identity',
    'uport',
    # What are the tools/libaries/framework you use
    'typedoc',
    'web3js',
    'test-rpc',
    'eth.js',
    'solc',
    'minimetoken',
    'visual_studio_code',
    'docker',
    'travis',
    'github',
    'rinkeby',
    'redux',
    'ethereumjs-blockstream',
    'keythereum',
    'sublime',
    'emacs',
    'vim',
    'bignumber',
    'blockies',
    'mm',
    'mythril',
    'mithril',
    'jetbrains',
    'jquery',
    'evmlab',
    'solhint',
    'ethgasreporter',
    'mocha',
    'oyente',
    'manticore',
    'rattle',
    'antlr',
    'vs_code',
    'typescript',
    'nodejs',
    'etherscan',
    'web3py',
    'django',
    'bootstrap',
    'evm_lab',
    'ethers.js',
    'google_cloud',
    'eth.js',
    'intellij',
    'spank',
    'zeroclient',
    'postgresql',
    'ethers.cli',
    'ethers.build',
    'swarm',
    'eth-keyring-controller',
    'ethjs-utils',
    'openostbase',
    'parityjs',
    'jupiter_notebook',
    'jupyter',
    '0x.js'
    'kubernetes',
    'github',
    'webpack',
    'brave',
    'libp2p',
    'devp2p',
    'atom',
    'apm',
    'aragon_pakage_manager',
    'pprof',
    'wyvern',
    'bignum',
    'numeral',
    'chai',
    'solcoverage',
    # How do you handle testing?
    'surya',
    # How do you handle smart contract verification and security?
    'professional',
    'argus',
    'ci',
    'human_thinking',
    'linting_rule',
    'linting',
    'linter',
    're-entrancy',
    'reentrancy',
    'vulnerability',
    'community_auditing',
    'bounty_program',
    'invariant',
    'unit_test',
    'unit_testing',
    'code_review',
    'quantstamp',
    'crappy_process',
    'spec',
    'edge_case',
    'corner_case',
    'intentional_break',
    'crowdfunding',
    'defensive_code',
    'risk',
    'mistake',
    'isolate',
    'don_hold_eth',
    'escape_hatch',
    'library',
    'compiler',
    'tester',
    'tooling',
    'error',
    'concept',
    'economics',
    'funcitonal_test',
    'fuzzing',
    'ethers.build',
    'yuet',
    'tdd',
    'internal_audit',
    'checklist',
    'integration_test',
    'consensys_best_practices',
    'subreddit',
    'external_audit',
    'bug_bounty',
    'security_analysis',
    'overflow',
    'assumption',
    # Other bounties?
    'state_watching',
    'nft',
    'openssl',
    'orderwatcher',
    'consensus-building',
    'vocal',
    'good_idea',
    'wrong_abstraction',
    'tokenized_equity',
    'neufund',
    'eurotoken',
    'C++_dev',
    'payment',
    'desktop_wallet',
    'web_page',
    'open-source_explorer',
    'decentralized',
    'erc223',
    'watcher',
    'solidity_debugger',
    'trusted_relay_chain',
    'side_chain',
    'sidechain',
    'deposit',
    'lock',
    'escrow',
    'talented_people',
    'web_developer',
    'software_engineer',
    'recruiting',
    'gas_profiling',
    'gas_cost_estimation',
    'stack',
    'limit',
    'slot',
    'iulia',
    'technical_writer',
    'improve_documentation',
    'good_ide',
    'omkara',
    'smart_contract_security',
    'solidity_contract',
    'incentive_structure',
    'reviewer',
    'maintainer',
    'bug-bounty',
    'dev_grant',
    'ethprize',
    'scaffolding_tool',
    'block_explorer',
    'scaling',
    'proof-of-authority',
    'security_team',
    'hire_people',
    'hiring_people', # should be catched by lemmatizer
    'full_time',
    'ethereum_query_language',
    'websocket',
    'deploy',
    'heroku',
    'deployment_tool',
    'manipulate_strings',
    'educate_people',
    'stay_safe',
    'tutorial',
    'write-up',
    'how_things_work',
    'resource',
    'phishing',
    'airdrop',
    'private_key',
    'onboard',
    'video',
    'infographic',
    'computational_analysis',
    'cost',
    'wasm',
    'black-box_test',
    'ide',
    'dependency',
    'code_coverage',
    'etherscan_verification',
    'tracing_api',
    'common_format',
    'tracing_rpc_call',
    'quickblocks',
    'swarm_integration',
    'coverage_tool',
    'ledger_support',
    'unification',
    'testnet',
    'type',
    'safety',
    'compiled_language',
    'use-case',
    'explain',
    'data_model',
    'interactive_demo',
    'upgradeability',
    'gas_cost',
    'scheduler',
    'compiler',
    'token_curation_market',
    # What tools don’t exist at the moment?
    'prettier',
    'breakpoint',
    'log',
    'slack',
    'estimate_gas',
    'software_development',
    'issue',
    'test_suite',
    'better_querying',
    'query',
    'good_wiki',
    'stack_visualizer',
    'evmlab',
    'execute',
    'trace',
    'step_through',
    'parity_bridge',
    'interoperability',
    'standardization',
    'bridge',
    'data_analytics',
    'reconciliation',
    'analytics',
    'data_implicit',
    'decentralized_logging',
    'automated_updatable_smart_contract',
    'ethmix',
    'syntax_highlighting',
    'event',
    'lightweight_user_interface',
    'subscribe',
    'blockchain_state',
    'bamboo',
    'vyper',
    'serpent',
    'rollback_transaction',
    'roll_back_transaction',
    'versioning',
    'type_system',
    'simulation_tool',
    'open_source',
    'storage_platform',
    'solidity_interpreter',
    'fraud_detection',
    'proxy_account',
    'interactive_debugger',
    'stress-test',
    'cryptoeconomics',
    'game_theory',
    'drag_and_drop',
    'builder',
    'user_experience',
    'quick_validation'
    }

## Proof of Concept

In [8]:
stop_words = nlp.Defaults.stop_words

def vocab_extraction(txts):
    # Transform the serie into a list of list of words,
    # Remove stopwords at the same time
    cleaned = []
    for idx, txt in txts.iteritems():
        # Remove stopwords
        cleaned += [[word for word in simple_preprocess(txt, deacc = True) if word not in stop_words]]
    
    # Build bigrams and trigrams
    bigrams = Phraser(Phrases(cleaned, min_count=1, threshold=1))
    trigrams = Phraser(Phrases(bigrams[cleaned], threshold=1))

    # Now create the bag of words with the new trigrams
    cleaned = [trigrams[bigrams[txt]] for txt in cleaned]
    
    # Lemmatization - TODO improve
    lemmatized = []
    for txt in cleaned:
        if txt == []:
            lemmatized += set()
        else:
            doc = nlp(" ".join(txt))
            # token.lemma_ has a tendency to replace 'ethereumjs' by 'ethereumj'
            lemmatized += [set(token.lemma_ for token in doc if (
                token.lemma_ in eth_keywords or token.lower_ in eth_keywords
                )
            )]
    
    return lemmatized

In [9]:
vocab_extraction(df['who_what'])

[{'contract',
  'decentralized_exchange',
  'design',
  'event',
  'gas',
  'network',
  'orderwatcher',
  'protocol',
  'smart_contract',
  'transaction'},
 {'blockchain', 'error', 'ethereum', 'intellij', 'truffle', 'usability'},
 {'api',
  'client',
  'contract',
  'mew',
  'proof_stake',
  'protocol',
  'raiden',
  'signature',
  'smart_contract',
  'token'},
 {'adchain',
  'consensy',
  'deploy',
  'design',
  'developer',
  'ethereum',
  'onboarding',
  'tcr',
  'token',
  'type'},
 {'ambisafe',
  'blockchain',
  'bounty',
  'client',
  'contract',
  'deploy',
  'deployment',
  'devop',
  'gas',
  'issue',
  'javascript',
  'parity',
  'platform',
  'scalable',
  'smart_contract',
  'test',
  'token',
  'token_contract',
  'truffle',
  'tx'},
 {'aragon',
  'aragon_o',
  'contract',
  'design',
  'ethereum',
  'network',
  'organization',
  'permission',
  'platform',
  'resource',
  'rinkeby',
  'ui'},
 {'acl',
  'aragon',
  'backend',
  'contract',
  'dapp',
  'database',
  'dece

In [10]:
vocab_extraction(df['tooling'])

[{'contract',
  'documentation',
  'ethereumj',
  'ethpm',
  'open_source',
  'smart_contract',
  'solidity',
  'truffle',
  'typedoc',
  'versioning'},
 {'client', 'raiden'},
 {'ganache', 'javascript', 'truffle'},
 {'apm',
  'aragon',
  'contract',
  'dapphub',
  'deploy',
  'design',
  'ens',
  'error',
  'ethereum',
  'ethpm',
  'event',
  'evm',
  'ganache',
  'github',
  'ipfs',
  'issue',
  'javascript',
  'log',
  'minimetoken',
  'parity',
  'smart_contract',
  'solc',
  'solidity',
  'test',
  'tool',
  'tooling',
  'truffle',
  'ui',
  'websocket'},
 {'augur',
  'ci',
  'contract',
  'dependency',
  'deploy',
  'docker',
  'error',
  'ethereumj',
  'geth',
  'github',
  'integration_test',
  'keythereum',
  'library',
  'parity',
  'rinkeby',
  'rpc',
  'test',
  'testrpc',
  'travis',
  'truffle',
  'typescript',
  'ui'},
 {'augur',
  'contract',
  'emacs',
  'ide',
  'integration',
  'remix',
  'smart_contract',
  'stack',
  'static_analysis',
  'sublime',
  'syntax_highlig

In [11]:
vocab_extraction(df['testing'])

[{'augur',
  'contract',
  'ethereumjs',
  'governance',
  'integration',
  'issue',
  'neufund',
  'open',
  'security',
  'signature',
  'smart_contract',
  'testrpc',
  'token'},
 {'truffle', 'unit_testing'},
 {'github', 'security', 'smart_contract', 'test'},
 {'audit', 'ci', 'compiler', 'contract', 'issue', 'security', 'solc'},
 {'investor', 'js', 'stack', 'test', 'test_suite', 'truffle'},
 {'deploy', 'rinkeby', 'testrpc', 'truffle'},
 {'assumption',
  'bounty',
  'contract',
  'ethereum',
  'formal_verification',
  'js',
  'mythril',
  'solidity',
  'test',
  'test_suite',
  'truffle',
  'unit_test',
  'vm'},
 {'library', 'travis', 'unit_test'},
 {'resource'},
 {'mythril', 'remix', 'surya'},
 {'audit', 'community', 'contract'},
 {'library', 'solidity', 'test'},
 {'issue', 'js', 'react'},
 {'geth', 'mocha', 'signature', 'test', 'testrpc'},
 {'contract', 'deploy', 'event', 'gas', 'truffle'},
 {'integration', 'test', 'unit_test'},
 {'deploy', 'metamask', 'remix', 'truffle'},
 {'gitco

## Formatting for JSON

In [12]:
def to_escaped_html(txt):
    paragraphs = [f"<p>{named_entities(line)}</p>" for line in txt.splitlines()]
    return ''.join(paragraphs)

In [13]:
def genTransformer(field, question):
    def resultProc(df):
        # Transform the serie into a list of list of words,
        # Remove stopwords at the same time
        cleaned = []
        for idx, txt in df[field].iteritems():
            # Remove stopwords
            cleaned += [[word for word in simple_preprocess(txt, deacc = True) if word not in stop_words]]

        # Build bigrams and trigrams
        bigrams = Phraser(Phrases(cleaned, min_count=1, threshold=1))
        trigrams = Phraser(Phrases(bigrams[cleaned], threshold=1))

        # Now create the bag of words with the new trigrams
        cleaned = [trigrams[bigrams[txt]] for txt in cleaned]

        # Lemmatization - TODO improve
        for idx, txt in enumerate(cleaned):
            if txt == []:
                df[field][idx] = {
                'question': question,
                'answer': to_escaped_html(df[field][idx]),
                'keywords': set()
            }
            else:
                doc = nlp(" ".join(txt))
                df[field][idx] = {
                    'question': question,
                    'answer': to_escaped_html(df[field][idx]),
                    'keywords': set(token.lemma_ for token in doc if (
                                        token.lemma_ in eth_keywords or token.lower_ in eth_keywords
                                ))
                }
    return resultProc

In [14]:
testing_proc = genTransformer('testing', 'How do you handle testing?')

In [15]:
testing_proc(df)

In [16]:
df['testing'][0]

{'question': 'How do you handle testing?',
 'answer': '<p>Testrpc has gotten better &ndash;&ndash;</p><p>Increases the speed at which you can dev.</p><p>Blockstream is really cool. Written by Mica (from Augur?), under the EthereumJS lib.</p><p>Good dealing with block re-orgs and changes in state.</p><p>Continuous Integration:</p><p>Doesn&rsquo;t really work for smart contracts.</p><p>We just use testrpc.</p><p>Neufund was different than 0x:</p><p>Neufund was a trustee for tokens &ndash;&ndash; it was centralized by definition.</p><p>This made it simpler: I could create &ldquo;open-ended&rdquo; contracts &ndash;&ndash; a switch statement, that the default would send to another proxy contract.</p><p>But wouldn&rsquo;t be able to do this with 0x &ndash;&ndash; e.g. if we wanted to add Shnorr signatures &ndash;&ndash; because it&rsquo;s a governance / security issues.</p>',
 'keywords': {'augur',
  'contract',
  'ethereumjs',
  'governance',
  'integration',
  'issue',
  'neufund',
  'open

In [17]:
who_what_proc = genTransformer('who_what', 'Who are you and what are you working on?')
tooling_proc = genTransformer('tooling', 'What are the tools/libraries/frameworks you use?')
smart_contract_proc = genTransformer('smart_contract', 'How do you handle smart contract verif & security?')
other_bounties_proc = genTransformer('bounties', 'Other bounties?')
missing_tools_proc = genTransformer('missing_tools', 'What tools don’t exist at the moment?')

In [18]:
who_what_proc(df)
tooling_proc(df)
smart_contract_proc(df)
other_bounties_proc(df)
missing_tools_proc(df)

In [19]:
df['who_what'][0]

{'question': 'Who are you and what are you working on?',
 'answer': '<p>0x - Decentralized exchange protocol. It is a set of smart contracts that handle standard transactions in an order format.</p><p>The key advantage here is that these smart contracts leads to shared liquidity</p><p>We also achieve increased security and network liquidity through our architecture.</p><p>We launched the network in August 2017.</p><p>It works, but is V1. We are working on V2 which helps people build hybrid stations, supports new token standards, and makes the contract more gas efficient.</p><p>V1 only allows humans to create orders; V2 will allow new smart contract to create orders. This is HUGE.</p><p>Notes:</p><p>We&rsquo;re building the protocol; designing the standard; building dev tools for anyone to build a decentralized exchange.</p><p>Orders may become invalid for any reason:</p><p>Order being filled</p><p>Griefing attacks</p><p>We&rsquo;ve now built an OrderWatcher that keeps relayers updated 

In [20]:
df[['name', 'who_what', 'tooling', 'testing', 'smart_contract', 'bounties', 'missing_tools']].to_json('interviews.json', orient = 'index')